### **In the following notebook, I have outlined a generic way to use the pytesseract library to read PDF documents and automatically transform them into DataFrames.**

**Only the information regarding academic background and skills is covered in the example, but the entire document can be extracted in the same manner.**

**The step-by-step explanation of what is being done in the code is in the form of comments.**


In [1]:
#Importa as bibliotecas

from pdf2image import convert_from_path
from pytesseract import pytesseract, image_to_string
from PIL import Image
from numpy import array, where
import os
from datetime import datetime, timedelta
from re import search, findall
from shutil import copytree, rmtree
from gc import collect
import cv2
import re
import pandas as pd
from IPython.display import display_html
from itertools import chain,cycle


#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)
#pd.set_option('max_colwidth', -1)

pytesseract.tesseract_cmd = (r"C:\Program Files\Tesseract-OCR\tesseract.exe")

In [2]:
#Funções


#Convertendo o PDF para imagem
def convert_pdf_to_img(pdf_file):
    image = convert_from_path(pdf_file, poppler_path=r'C:\Users\henri\Downloads\poppler\Release-24.02.0-0\poppler-24.02.0\Library\bin', fmt='jpeg', dpi=350)
    return image

#Convertendo a imagem para texto
def convert_image_to_text(file):
    text = image_to_string(file, config='--psm 6')
    return text

#Encontrando o indice da palavra
def contem_palavra(sublista, palavra):
    return any(palavra in frase for frase in sublista)

#Fazer o display de 2 DataFrames juntos 
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df, title in zip(args, chain(titles, cycle(['</br>']))):
       html_str += '<div style="display: inline-block; vertical-align: top; margin: 10px; text-align: center;">'
       html_str += f'<h2 style="text-align: center;">{title}</h2>'
       html_str += df.to_html().replace('table', 'table style="display:inline"')
       html_str += '</div>'
    display_html(f'<div style="text-align: center;">{html_str}</div>', raw=True)

#Delimitadores das habilidades
def dividir_string(s):
    delimitadores = ['1.', '2.', '3.','4.', '5.', '6.','7.', '7 .', '8.', '9.','10.', '11.', '12.']
    for delimitador in delimitadores:
        s = s.replace(delimitador, ',')
    return [item.strip() for item in s.split(',') if item.strip()]

#lendo o arquivo
def bens_direitos(arquivo):

    image = convert_pdf_to_img(arquivo)

    formacao = []
    habilidades = []
    lista_dividida = []
    for i, img in enumerate(image):
        final_text = ""
        image_array = array(img)

        actual_text = convert_image_to_text(image_array)
        actual_text = actual_text.lower()

        linhas_nomes = actual_text.split('\n')
        lista_de_vetores = [linha.split(',') for linha in linhas_nomes]
        
        #Formação academica
        palavra_procurada = 'formacao academica'
        indice = next((i for i, sublista in enumerate(lista_de_vetores) if contem_palavra(sublista, palavra_procurada)), None)
        if indice is not None:
                while True:
                        indice += 1
                        if indice >= len(lista_de_vetores):  
                                break

                        if any("cursos adicionais" in frase for frase in lista_de_vetores[indice]):
                                break
                        formacao.append(lista_de_vetores[indice])


        #Habilidades
        try:
                palavra_procurada = 'habilidades'
                indice_2 = next((i for i, sublista in enumerate(lista_de_vetores) if contem_palavra(sublista, palavra_procurada)), None)
                while True:
                        indice_2 += 1
                        if indice_2 >= len(lista_de_vetores):  
                                break

                        if lista_de_vetores[indice_2] == ['']:
                                break

                        habilidades.append(lista_de_vetores[indice_2])
                        
        except:
                pass

        
        for sublista in habilidades:
                for item in sublista:
                        lista_dividida.extend(dividir_string(item))



    #DataFrame da formação
    dados_flat = [item[0] for item in formacao]
    dados_estruturados = [
            dados_flat[:3],
                    dados_flat[3:]   
    ]
    df_formacao = pd.DataFrame(dados_estruturados, columns=['Formação', 'Local', 'Data'])


    #DataFrame das habilidades
    df_habilidades = pd.DataFrame(lista_dividida, columns=['Habilidade'])

                            
    return df_formacao, df_habilidades



In [3]:
#Caminho do arquivo do currículo
arquivo = "C:/Users/henri/Downloads/Currículo_do_Henrique (1).pdf"

In [4]:
df_formacao, df_habilidades = bens_direitos(arquivo)

In [5]:
display_side_by_side(df_formacao, df_habilidades, titles=['Formação','Habilidades'])

Formação 
 
 
 
 Formação 
 Local 
 Data 
 
 
 
 
 0 
 - bacharelado em estatistica 
 universidade federal de uberlandia 
 (03/2020 - 04/2024) 
 
 
 1 
 - mestrado em intelig&ncia artificial 
 universidade federal de uberlandia 
 (07/2024 - 07/2027) 
 
 
 Habilidades 
 
 
 
 Habilidade 
 
 
 
 
 0 
 python 
 
 
 1 
 estatistica 
 
 
 2 
 linguagem r 
 
 
 3 
 andlise de dados 
 
 
 4 
 inglés 
 
 
 5 
 oftice 
 
 
 6 
 sql 
 
 
 7 
 power bl 
 
 
 8 
 sas 1 
 
 
 9 
 gcp 
 
 
 10 
 modelagem estatistica 1 
 
 
 11 
 aws